<div style="text-align:center;font-size:22pt; font-weight:bold;color:white;border:solid black 1.5pt;background-color:#1e7263;">
    TensorBoard Callback Overview
</div>

In [1]:
# ======================================================================= #
# Course: Deep Learning Complete Course (CS-501)
# Author: Dr. Saad Laouadi
# Institution: Quant Coding Versity Academy
# Date: December 25, 2024
#
# ==========================================================
# Lesson: Learning Rate Adaptation with ReduceLROnPlateau
#         Synthetic Data Example
# ==========================================================
# ## Learning Objectives
# This example will enable you to:
# 1. Create synthetic data for learning rate adaptation
# 2. Implement ReduceLROnPlateau callback
# 3. Monitor learning rate changes during training
# 4. Visualize the impact of learning rate reduction
# 5. Compare training with and without adaptive learning rates
# =======================================================================
#          Copyright © Dr. Saad Laouadi 2024
# =======================================================================

In [2]:
# ============================================================================ #
#                         Environment Path Configuration                       #
# ============================================================================ #
#
# Purpose:
#   Configure the system PATH to use Python executables from the active virtual 
#   environment instead of global installations.
#
# Usage:
#   1. First verify if configuration is needed by running: !which python
#   2. If the output shows the global Python installation rather than your 
#      virtual environment, execute this configuration block
#
# Note:
#   This configuration is typically only needed for JupyterLab Desktop or 
#   similar standalone installations. Web-based JupyterLab or properly 
#   configured environments should not require this adjustment.
# ============================================================================ #

import os
import sys

env_path = os.path.dirname(sys.executable)
os.environ['PATH'] = f"{env_path}:{os.environ['PATH']}"

In [3]:
# ==================================================== #
#        Load Required Libraries
# ==================================================== #
import shutil
from datetime import datetime
import io

# Disable Metal API Validation
os.environ["METAL_DEVICE_WRAPPER_TYPE"] = "0"  

import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# import tensorflow
import tensorflow as tf

# Set styling for better visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("="*72)

%reload_ext watermark
%watermark -a "Dr. Saad Laouadi" -u -d -m

print("="*72)
print("Imported Packages and Their Versions:")
print("="*72)

%watermark -iv
print("="*72)

# Global Config
RANDOM_STATE = 101

Author: Dr. Saad Laouadi

Last updated: 2025-01-09

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 24.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Imported Packages and Their Versions:
psutil    : 5.9.0
sys       : 3.11.10 (main, Oct  3 2024, 02:26:51) [Clang 14.0.6 ]
numpy     : 1.26.4
tensorflow: 2.16.2
seaborn   : 0.13.2
sklearn   : 1.5.1
pandas    : 2.2.2
matplotlib: 3.9.2



In [4]:
# =============================================================================
# Educational Utility Function: Directory Cleanup
# =============================================================================
#
# Purpose:
#   This utility function is designed for educational purposes in the context
#   of programming tutorials and exercises. It provides a safe way to clean up
#   working directories during course activities.
#
# Usage Context:
#   - Instructors: Useful for preparing and resetting educational content
#   - Students: Helpful when experimenting with code examples
#   - Learning Environment: Supports clean workspace management
#
# Note:
#   This function is intended for educational environments only and should be
#   used with caution as it permanently deletes directory contents.
# =============================================================================

def cleanup_directory(directory_path):
    """
    Deletes the specified directory and all its contents.
    Args:
        directory_path (str): Path to the directory to delete.
    """
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        shutil.rmtree(directory_path)
        print(f"Directory '{os.path.basename(directory_path)}' deleted successfully.")
    else:
        print(f"Directory '{os.path.basename(directory_path)}' does not exist or is not a directory.")
        
        
# tensorboard cleanup        
def cleanup_tensorboard():
    """
    Cleanup TensorBoard processes professionally
    """
    # Find and terminate TensorBoard processes
    for proc in psutil.process_iter(['pid', 'name']):
        try:
            # Look for tensorboard processes
            if 'tensorboard' in proc.info['name'].lower():
                # Terminate gracefully
                process = psutil.Process(proc.info['pid'])
                process.terminate()
                print(f"TensorBoard process {proc.info['pid']} terminated gracefully")
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass

---

### Requirements

This notebook needs the following tools to run without errors. You can install the next tools from within a notebook cell without leaving this working environment. 

I recommend running this first

```python
import os
import sys

env_path = os.path.dirname(sys.executable)
os.environ['PATH'] = f"{env_path}:{os.environ['PATH']}"
```

Check the cell {[2]} above for more information. 


1. **The `psutil` package**: You might need to install `psutil` if you haven't already by running the next command:

```python
!pip install psutil
```

2. **The `tensorboard` package**: You may install `tensorboard` by running this command

```bash
!pip install tensorboard
```

If the somethind did not go as expected, you may need to check this [tensorboard overview](07-04-00-callbacks-tensorboard-overview.ipynb) for more details on to setup your environment to use `tensorboard`.

### Raodmap
1. Load and process House prices Data
2. Create Model with `tensorboard` callback
4. Use and interpret the results on tensorboard visualization tool
5. Use more advanced model with multiple callbacks.

In [5]:
# ================================================================= #
#          California House Prices Regression Project
# ================================================================= #

# A function to load and process the data
def load_and_preprocess_data(random_state=0):
    """Load and preprocess the California Housing dataset"""
    # Load data
    housing = fetch_california_housing()
    X, y = housing.data, housing.target
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=random_state
    )
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test, housing.feature_names

# Create a function to build and compile the model
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(8,)),  # California Housing has 8 features
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)  # Single output for regression
    ])
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    
    model.compile(
        optimizer=optimizer,
        loss='mse',  # Mean Squared Error for regression
        metrics=[
            'mae',  # Mean Absolute Error
            'mse'   # Mean Squared Error
        ]
    )
    return model

In [6]:
# Create directory for logs with meaningful name
model_name = "california_housing"
log_dir = os.path.join("logs", "fit", model_name, datetime.now().strftime('%Y%m%d-%H%M%S'))
os.makedirs(log_dir, exist_ok=True)

# Create TensorBoard callback with modern configuration
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,              # Enable histogram computation each epoch
    write_graph=True,
    write_images=False,            # Usually not needed, can make logs very large
    update_freq='epoch',
    profile_batch=(5, 10)          # Profile batches 5 through 10
)

In [7]:
# Load TensorBoard extension
%load_ext tensorboard

# Clear previous logs (optional)
!rm -rf logs/fit

# Create the model
model = create_model()

In [8]:
# Load and preprocess data
X_train, X_test, y_train, y_test, feature_names = load_and_preprocess_data()
        
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=50,              # Increased epochs for better training
    batch_size=32,
    validation_split=0.2,
    callbacks=[tensorboard_callback],
    verbose=1
)

# Launch TensorBoard
%tensorboard --logdir logs/fit

Epoch 1/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 2.3288 - mae: 1.0588 - mse: 2.3288 - val_loss: 0.4368 - val_mae: 0.4725 - val_mse: 0.4368
Epoch 2/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.4471 - mae: 0.4710 - mse: 0.4471 - val_loss: 0.3766 - val_mae: 0.4515 - val_mse: 0.3766
Epoch 3/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.3942 - mae: 0.4502 - mse: 0.3942 - val_loss: 0.3524 - val_mae: 0.4314 - val_mse: 0.3524
Epoch 4/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.3674 - mae: 0.4318 - mse: 0.3674 - val_loss: 0.3395 - val_mae: 0.4236 - val_mse: 0.3395
Epoch 5/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.3706 - mae: 0.4302 - mse: 0.3706 - val_loss: 0.3383 - val_mae: 0.4316 - val_mse: 0.3383
Epoch 6/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.3529 - mae: 0.4126 - mse: 0.3529 - val_loss: 0.3358 - val_mae: 0.4028 - val_mse: 0.3358
Epoch 7/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.3374 - mae: 0.4126 - mse: 0.3374 - val

Reusing TensorBoard on port 6006 (pid 99236), started 0:36:01 ago. (Use '!kill 99236' to kill it.)

In [9]:
# Use before starting a new TensorBoard session
cleanup_tensorboard()

# Clear any previous logs
!rm -rf california_housing/logs/fit

### Running Advanced Model 

In [10]:
def create_advanced_model():
    """Create advanced model architecture for california house prices project."""
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(8,)),
        
        # First block with residual connection
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        # Second block with residual connection
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        
        # Third block
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.1),
        
        # Output layer
        tf.keras.layers.Dense(1)
    ])
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    
    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=[
            'mae',
            tf.keras.metrics.RootMeanSquaredError(name='rmse')
        ]
    )
    return model

# Load and preprocess data
X_train, X_test, y_train, y_test, feature_names = load_and_preprocess_data()

# Set up callbacks
def create_callbacks(model_name="california_housing"):
    """Create all callbacks for training"""
    
    # Create directories for logs and checkpoints
    base_dir = "training_logs"
    log_dir = os.path.join(base_dir, "logs", model_name, datetime.now().strftime('%Y%m%d-%H%M%S'))
    checkpoint_dir = os.path.join(base_dir, "checkpoints", model_name)
    os.makedirs(log_dir, exist_ok=True)
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    callbacks = [
        # TensorBoard callback
        tf.keras.callbacks.TensorBoard(
            log_dir=log_dir,
            histogram_freq=1,
            write_graph=True,
            write_images=False,
            update_freq='epoch',
            profile_batch=(5, 10)
        ),
        
        # Model checkpoint callback
        tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(checkpoint_dir, 'model_{epoch:02d}-{val_loss:.4f}.keras'),
            save_best_only=True,
            monitor='val_loss',
            mode='min',
            verbose=1
        ),
        
        # Early stopping callback
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        
        # Reduce LR on plateau callback
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-6,
            verbose=1
        )
    ]
    
    return callbacks

In [11]:
# Clean up previous TensorBoard sessions
%reload_ext tensorboard
!rm -rf training_logs/logs/*

# Create and train the model
model = create_advanced_model()
callbacks = create_callbacks()

In [12]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/100
413/413 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 3.6513 - mae: 1.5217 - rmse: 1.8832
Epoch 1: val_loss improved from inf to 0.54148, saving model to training_logs/checkpoints/california_housing/model_01-0.5415.keras
413/413 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - loss: 3.6478 - mae: 1.5208 - rmse: 1.8823 - val_loss: 0.5415 - val_mae: 0.5346 - val_rmse: 0.7359 - learning_rate: 0.0010
Epoch 2/100
411/413 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.8522 - mae: 0.6983 - rmse: 0.9227
Epoch 2: val_loss improved from 0.54148 to 0.43051, saving model to training_logs/checkpoints/california_housing/model_02-0.4305.keras
413/413 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - loss: 0.8517 - mae: 0.6980 - rmse: 0.9224 - val_loss: 0.4305 - val_mae: 0.4641 - val_rmse: 0.6561 - learning_rate: 0.0010
Epoch 3/100
411/413 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.6371 - mae: 0.5989 - rmse: 0.7981
Epoch 3: val_loss improved from 0.43051 to 0.40270, saving model to training_logs/checkpoints/california_hou

In [15]:
# Launch TensorBoard
%tensorboard --logdir training_logs/logs

In [ ]:
# Save Training Plots
def save_training_plots(history, save_dir="training_logs/plots"):
    """
    Save training history plots
    
    Args:
        history: Keras history object containing training metrics
        save_dir: Directory to save the plot files
    
    Returns:
        None
    """
    if not history or not hasattr(history, 'history'):
        raise ValueError("Invalid history object")
        
    os.makedirs(save_dir, exist_ok=True)
    
    # Plot loss
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    if 'val_loss' in history.history:
        plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(save_dir, 'loss_plot.png'))
    plt.close()
    
    # Plot metrics
    for metric in ['mae', 'rmse']:
        if metric in history.history:
            plt.figure(figsize=(10, 6))
            plt.plot(history.history[metric], label=f'Training {metric.upper()}')
            val_metric = f'val_{metric}'
            if val_metric in history.history:
                plt.plot(history.history[val_metric], 
                        label=f'Validation {metric.upper()}')
            plt.title(f'Model {metric.upper()}')
            plt.xlabel('Epoch')
            plt.ylabel(metric.upper())
            plt.legend()
            plt.grid(True)
            plt.savefig(os.path.join(save_dir, f'{metric}_plot.png'))
            plt.close()

# Use after training
save_training_plots(history)